Hey, I'm Jobert Gutierrez and hereafter you'll find the logic and code used to answer the fourth assignment in the program Data Engineering Zoomcamp offered by Data Talks Club.

# __Module 4 Homework: Analytics Engineering__

For this homework, you will need the following datasets:

- [Green Taxi dataset (2019 and 2020)](https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/green)
- [Yellow Taxi dataset (2019 and 2020)](https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/yellow)
- [For Hire Vehicle dataset (2019)](https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/fhv)

Before you start,

1. Make sure you, at least, have them in GCS with a External Table OR a Native Table - use whichever method you prefer to accomplish that (Workflow Orchestration with pandas-gbq, dlt for gcs, dlt for BigQuery, gsutil, etc)
2. You should have exactly `7,778,101` records in your Green Taxi table
3. You should have exactly `109,047,518` records in your Yellow Taxi table
4. You should have exactly `43,244,696` records in your FHV table
5. Build the staging models for green/yellow as shown in [here](https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/04-analytics-engineering/taxi_rides_ny/models/staging)

Note: If you don't have access to GCP, you can spin up a local Postgres instance and ingest the datasets above

### __Question 1.Understanding dbt model resolution__
Provided you've got the following sources.yaml?

In [ ]:
version: 2

sources:
  - name: raw_nyc_tripdata
    database: "{{ env_var('DBT_BIGQUERY_PROJECT', 'dtc_zoomcamp_2025') }}"
    schema:   "{{ env_var('DBT_BIGQUERY_SOURCE_DATASET', 'raw_nyc_tripdata') }}"
    tables:
      - name: ext_green_taxi
      - name: ext_yellow_taxi

with the following env variables setup where dbt runs:

In [ ]:
export DBT_BIGQUERY_PROJECT=myproject
export DBT_BIGQUERY_DATASET=my_nyc_tripdata

What does this .sql model compile to?

In [ ]:
select * 
from {{ source('raw_nyc_tripdata', 'ext_green_taxi' ) }}

--select * from dtc_zoomcamp_2025.raw_nyc_tripdata.ext_green_taxi <br>
--select * from dtc_zoomcamp_2025.my_nyc_tripdata.ext_green_taxi <br>
--select * from myproject.raw_nyc_tripdata.ext_green_taxi <br>
--select * from myproject.my_nyc_tripdata.ext_green_taxi <br>
-- select * from dtc_zoomcamp_2025.raw_nyc_tripdata.green_taxi

### Answer: 
The indicated sql model compiles to `select * from myproject.raw_nyc_tripdata.ext_green_taxi`.

### __Question 2.dbt Variables & Dynamic Models__

Say you have to modify the following dbt_model (`fct_recent_taxi_trips.sql`) to enable Analytics Engineers to dynamically control the date range.

- In development, you want to process only the last 7 days of trips
- In production, you need to process the last 30 days for analytics


In [ ]:
select *
from {{ ref('fact_taxi_trips') }}
where pickup_datetime >= CURRENT_DATE - INTERVAL '30 days'

What would you change to accomplish that in a such way that command line arguments takes precedence over ENV_VARs, which takes precedence over DEFAULT value?

- Add `ORDER BY pickup_datetime DESC` and `LIMIT {{ var("days_back", 30) }}`
- Update the WHERE clause to `pickup_datetime >= CURRENT_DATE - INTERVAL '{{ var("days_back", 30) }}' DAY`
- Update the WHERE clause to `pickup_datetime >= CURRENT_DATE - INTERVAL '{{ env_var("DAYS_BACK", "30") }}' DAY`
- Update the WHERE clause to `pickup_datetime >= CURRENT_DATE - INTERVAL '{{ var("days_back", env_var("DAYS_BACK", "30")) }}' DAY`
- Update the WHERE clause to `pickup_datetime >= CURRENT_DATE - INTERVAL '{{ env_var("DAYS_BACK", var("days_back", "30")) }}' DAY`

### Answer: 
The right answer is __updating the WHERE clause to `pickup_datetime >= CURRENT_DATE - INTERVAL '{{ var("days_back", env_var("DAYS_BACK", "30")) }}' DAY`__ because this option:
- ensures prioritization If days_back is passed via --vars, 
- it falls back to env_var("DAYS_BACK", "30") if `days_back` isn't set, and 
- set the dafault value of 30 if neither is set.

### __Question 3.dbt Data Lineage and Execution__

Considering the data lineage below and that taxi_zone_lookup is the only materialization build (from a .csv seed file):

![homework_q2](homework_q2.png "Diagram")

Select the option that does NOT apply for materializing fct_taxi_monthly_zone_revenue:

- dbt run
- dbt run --select +models/core/dim_taxi_trips.sql+ --target prod
- dbt run --select +models/core/fct_taxi_monthly_zone_revenue.sql
- dbt run --select +models/core/
- dbt run --select models/staging/+

### Answer:
The right answer is `dbt run --select +models/core/dim_taxi_trips.sql+ --target prod` because this will not materialize the table in dev. It will create the table in production only.

### __Question 4.__
dbt Macros and Jinja
Consider you're dealing with sensitive data (e.g.: [PII](https://en.wikipedia.org/wiki/Personal_data)), that is only available to your team and very selected few individuals, in the `raw layer` of your DWH (e.g: a specific BigQuery dataset or PostgreSQL schema),

- Among other things, you decide to obfuscate/masquerade that data through your staging models, and make it available in a different schema (a `staging layer`) for other Data/Analytics Engineers to explore

- And optionally, yet another layer (`service layer`), where you'll build your dimension (dim_) and fact (fct_) tables (assuming the [Star Schema dimensional modeling](https://www.databricks.com/glossary/star-schema)) for Dashboarding and for Tech Product Owners/Managers

You decide to make a macro to wrap a logic around it:

In [ ]:
{% macro resolve_schema_for(model_type) -%}

    {%- set target_env_var = 'DBT_BIGQUERY_TARGET_DATASET'  -%}
    {%- set stging_env_var = 'DBT_BIGQUERY_STAGING_DATASET' -%}

    {%- if model_type == 'core' -%} {{- env_var(target_env_var) -}}
    {%- else -%}                    {{- env_var(stging_env_var, env_var(target_env_var)) -}}
    {%- endif -%}

{%- endmacro %}

And use on your staging, dim_ and fact_ models as:

In [ ]:
{{ config(
    schema=resolve_schema_for('core'), 
) }}

That all being said, regarding macro above, select all statements that are true to the models using it:

- Setting a value for DBT_BIGQUERY_TARGET_DATASET env var is mandatory, or it'll fail to compile
- Setting a value for DBT_BIGQUERY_STAGING_DATASET env var is mandatory, or it'll fail to compile
- When using core, it materializes in the dataset defined in DBT_BIGQUERY_TARGET_DATASET
- When using stg, it materializes in the dataset defined in DBT_BIGQUERY_STAGING_DATASET, or defaults to DBT_BIGQUERY_TARGET_DATASET
- When using staging, it materializes in the dataset defined in DBT_BIGQUERY_STAGING_DATASET, or defaults to DBT_BIGQUERY_TARGET_DATASET

### Answer:
The right options are: 
- `Setting a value for DBT_BIGQUERY_TARGET_DATASET env var is mandatory, or it'll fail to compile` because the macro always references env_var(target_env_var), which is DBT_BIGQUERY_TARGET_DATASET.

- `When using core, it materializes in the dataset defined in DBT_BIGQUERY_TARGET_DATASET` because if the sets `model_type == 'core'`, the macro resolves to `{{- env_var(target_env_var) -}}`.

- `When using staging, it materializes in the dataset defined in DBT_BIGQUERY_STAGING_DATASET, or defaults to DBT_BIGQUERY_TARGET_DATASET` because if model_type is not 'core', the macro executes the else clause meaning that if DBT_BIGQUERY_STAGING_DATASET exists, it uses it or it falls back to DBT_BIGQUERY_TARGET_DATASET otherwise.

## __Serious SQL__
Alright, in module 1, you had a SQL refresher, so now let's build on top of that with some serious SQL.

These are not meant to be easy - but they'll boost your SQL and Analytics skills to the next level.
So, without any further do, let's get started...

You might want to add some new dimensions year (e.g.: 2019, 2020), quarter (1, 2, 3, 4), year_quarter (e.g.: 2019/Q1, 2019-Q2), and month (e.g.: 1, 2, ..., 12), extracted from pickup_datetime, to your fct_taxi_trips OR dim_taxi_trips.sql models to facilitate filtering your queries

### __Question 5.Taxi Quarterly Revenue Growth__
1. Create a new model `fct_taxi_trips_quarterly_revenue.sql`
2. Compute the Quarterly Revenues for each year for based on total_amount
3. Compute the Quarterly YoY (Year-over-Year) revenue growth
- e.g.: In 2020/Q1, Green Taxi had -12.34% revenue growth compared to 2019/Q1
- e.g.: In 2020/Q4, Yellow Taxi had +34.56% revenue growth compared to 2019/Q4

Considering the YoY Growth in 2020, which were the yearly quarters with the best (or less worse) and worst results for green, and yellow

- green: {best: 2020/Q2, worst: 2020/Q1}, yellow: {best: 2020/Q2, worst: 2020/Q1}
- green: {best: 2020/Q2, worst: 2020/Q1}, yellow: {best: 2020/Q3, worst: 2020/Q4}
- green: {best: 2020/Q1, worst: 2020/Q2}, yellow: {best: 2020/Q2, worst: 2020/Q1}
- green: {best: 2020/Q1, worst: 2020/Q2}, yellow: {best: 2020/Q1, worst: 2020/Q2}
- green: {best: 2020/Q1, worst: 2020/Q2}, yellow: {best: 2020/Q3, worst: 2020/Q4}

### Answer:
The best strategy to make an optimized a table in BigQuery using the fields of "tpep_dropoff_datetime" and "VendorID" is __Partition by tpep_dropoff_datetime and Cluster on VendorID__, because only one field can be selected to partition upon and a datetime field can reduce the amount of data of each partition more effectively than the vendors ID. The code used to create such table is:

In [ ]:
create or replace table `taxi-data-24-dtc.ny_taxi_25.yellow_taxi_partition_and_cluster`
partition by date(tpep_dropoff_datetime)
cluster by VendorID as 
select * from `taxi-data-24-dtc.ny_taxi_25.yellow_taxi_25_non_partition`

### __Question 6.__
Write a query to retrieve the distinct VendorIDs between tpep_dropoff_datetime 2024-03-01 and 2024-03-15 (inclusive)

Use the materialized table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 5 and note the estimated bytes processed. What are these values?

Choose the answer which most closely matches.

- 12.47 MB for non-partitioned table and 326.42 MB for the partitioned table <br>
- 310.24 MB for non-partitioned table and 26.84 MB for the partitioned table <br>
- 5.87 MB for non-partitioned table and 0 MB for the partitioned table <br>
- 310.31 MB for non-partitioned table and 285.64 MB for the partitioned table


### Answer:
The query used is:

In [ ]:
# For non partitioned table:
select distinct(VendorID) from `taxi-data-24-dtc.ny_taxi_25.yellow_taxi_25_non_partition`
where date(tpep_dropoff_datetime) between '2024-03-01' and '2024-03-15'

![Q6-a](Q6-a.png "Non partitioned query")

In [ ]:
# And for partitioned tables is:
select distinct(VendorID) from `taxi-data-24-dtc.ny_taxi_25.yellow_taxi_partition_and_cluster`
where date(tpep_dropoff_datetime) between '2024-03-01' and '2024-03-15'

![Q6-b](Q6-b.png "Partitioned table")

The option that properly reflects the amount of data is __310.24 MB for non-partitioned table and 26.84 MB for the partitioned table__.

### __Question 7.__
Where is the data stored in the External Table you created?

- Big Query <br>
- Container Registry <br>
- GCP Bucket <br>
- Big Table


### Answer:
The data is stored in a __GCP Bucket__ for the external table.

### __Question 8.__
It is best practice in Big Query to always cluster your data:

- True <br>
- False

### Answer:
Its false because it depends on the amount of data in the table. When its little data, the compute resources used for building the clusters overrides the benefits during queries. 